In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Define a function to preprocess the text
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Tokenize the text into individual words
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords from the text
    stopwords_list = stopwords.words('english')
    filtered_tokens = [token for token in tokens if token not in stopwords_list]
    
    # Apply stemming or lemmatization to the tokens
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Return the preprocessed text as a string
    return ' '.join(lemmatized_tokens)


In [19]:
# Load the two CSV files into separate DataFrames
posts_df = pd.read_csv('Top_posts.csv')
comments_df = pd.read_csv('Top_posts_comments.csv')

In [20]:
# Merge the two DataFrames on the post ID column
merged_df = pd.merge(posts_df, comments_df, on='post_id')


In [21]:
merged_df = merged_df.drop('flair_text', axis=1)
#dropped flair because it had missing values in it and I though it would be best to drop it

In [22]:
merged_df.dropna(subset=['comment'], inplace=True)
# we drop the missing values 

In [23]:
merged_df['preprocessed_comment'] = merged_df['comment'].apply(preprocess_text)


In [24]:
# Preprocess the data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(merged_df['preprocessed_comment'])
y = merged_df['subreddit']

In [25]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
# Train a logistic regression classifier on the training data
lr = LogisticRegression()
lr.fit(X_train, y_train)

C:\Users\Rahul\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
# Predict the subreddit categories of the validation data using the trained logistic regression classifier
y_val_pred = lr.predict(X_val)

In [28]:
# Calculate the accuracy of the classifier on the validation data
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation accuracy:", val_accuracy)

Validation accuracy: 0.7173776662484316
